In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MinMaxScaler
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np

# Preparing CIFAR10 Dataset for the models as CIFAR10 contains almost land vehicles data.
(X_train , Y_train), (X_test, Y_test) = cifar10.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

Using TensorFlow backend.


In [2]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), padding='same', kernel_initializer = 'lecun_normal', input_shape=X_train.shape[1:]))
model.add(keras.layers.Activation('elu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32, (3,3), padding='same', kernel_initializer='lecun_normal'))
model.add(keras.layers.Activation('elu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Dropout(0.2))
 
model.add(keras.layers.Conv2D(64, (3,3), padding='same',kernel_initializer='lecun_normal'))
model.add(keras.layers.Activation('elu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, (3,3), padding='same',kernel_initializer='lecun_normal'))
model.add(keras.layers.Activation('elu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Dropout(0.3))
 
model.add(keras.layers.Conv2D(128, (3,3), padding='same',kernel_initializer='lecun_normal' ))
model.add(keras.layers.Activation('elu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(128, (3,3), padding='same',kernel_initializer='lecun_normal'))
model.add(keras.layers.Activation('elu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Dropout(0.4))
 
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)       

In [3]:
datagen = ImageDataGenerator(rotation_range=15,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True)

datagen.fit(X_train)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics = ['accuracy'])

model.fit_generator(datagen.flow(X_train, Y_train, batch_size = 64), steps_per_epoch=X_train.shape[0] // 64,epochs=20, validation_data=(X_test, Y_test))


Epoch 1/20
781/781 [==============================] - 38s 49ms/step - loss: 1.8228 - accuracy: 0.4114 - val_loss: 1.3265 - val_accuracy: 0.5436
Epoch 2/20
781/781 [==============================] - 33s 42ms/step - loss: 1.2920 - accuracy: 0.5485 - val_loss: 1.1728 - val_accuracy: 0.6050
Epoch 3/20
781/781 [==============================] - 32s 41ms/step - loss: 1.1101 - accuracy: 0.6102 - val_loss: 0.9527 - val_accuracy: 0.6723
Epoch 4/20
781/781 [==============================] - 32s 41ms/step - loss: 1.0049 - accuracy: 0.6482 - val_loss: 0.9969 - val_accuracy: 0.6725
Epoch 5/20
781/781 [==============================] - 32s 41ms/step - loss: 0.9309 - accuracy: 0.6741 - val_loss: 0.8446 - val_accuracy: 0.7192
Epoch 6/20
781/781 [==============================] - 32s 41ms/step - loss: 0.8716 - accuracy: 0.6937 - val_loss: 0.8574 - val_accuracy: 0.7147
Epoch 7/20
781/781 [==============================] - 33s 42ms/step - loss: 0.8321 - accuracy: 0.7080 - val_loss: 0.7880 - val_accuracy:

In [4]:
model.evaluate(X_test, Y_test, batch_size=128)

10000/10000 [==============================] - 2s 207us/step


[0.5869816507339477, 0.807699978351593]

In [7]:
import cv2
import numpy as np

cap = cv2.VideoCapture("D:/Python Projects/Data/Videos/carstest.mp4")

ret, frame1 = cap.read()
i = 0
while cap.isOpened():
    ret, frame2 = cap.read()
    diff = cv2.absdiff(frame1, frame2)
    diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(diff, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilate = cv2.dilate(thresh, None, iterations = 3)
    contours, _ = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #cv2.drawContours(frame1, contours, -1, (0, 255, 0), 2)

    for contour in contours:
        if cv2.contourArea(contour) < 2000:
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        image_frame = np.zeros((32, 32, 3), np.float32)
        image_frame = frame1[y : y + h, x : x + w]
        image_frame = np.resize(image_frame, (1, 32, 32, 3))
        if(np.argmax(model.predict(image_frame)) == 1):
            cv2.putText(frame1, "AutoMobile", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
        cv2.putText(frame1, "Status : Movement", (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2, cv2.LINE_AA)
    
    cv2.imshow("Frame", frame1)
    frame1 = frame2
    if cv2.waitKey(40) == 27:
        break
    i += 1
    
cv2.destroyAllWindows()
cap.release()

In [ ]:
import cv2
import numpy as np

def reorder(points):
    points_new = np.zeros_like(points)
    points = np.reshape(points, (4, 2))
    add = point.sum(1)
    points_new[0] = points[np.argmin(add)]
    points_new[3] = points[np.argmax(add)]
    diff = np.diff(points, axis = 1)
    points_new[1] = points[np.argmax(diff)]
    points_new[2] = points[np.argmin(diff)]
    
    return points_new

def warpimage(image, points, w, h):
    reorder(points)
    point_1 = np.float32(points)
    point_2 = np.float32([[0, 0], [w ,0], [0, h], [w, h]])
    matrix = cv2.getPerspectiveTransform(point_1, point_2) 
    
    
def getContours(image, thresholds, minArea):
    finalcontours = []
    imagegray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imageblur = cv2.GaussianBlur(imagegray, (5, 5), 1)
    #_, mask = cv2.threshold(imageblur, 40, 255, cv2.THRESH_BINARY)
    edges = cv2.Canny(imageblur, thresholds[0], thresholds[1])
    #image_d_e = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8), iterations = 3)
    image_d_e = cv2.dilate(edges, np.ones((5,5), np.uint8), iterations = 3)
    image_d_e = cv2.erode(image_d_e, np.ones((5,5), np.uint8), iterations = 2)
    cv2.imshow("Proccesed Image", image_d_e)
    
    contours, _ = cv2.findContours(image_d_e, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for i in contours:
        area = cv2.contourArea(i)
        
        if area > minArea:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            bbox = cv2.boundingRect(approx)
            finalcontours.append((len(approx), area, approx, bbox, i))
        
    finalcontours = sorted(finalcontours, key = lambda x:x[1], reverse = True)
    
    for contour_detail in finalcontours:
        cv2.drawContours(image, [contour_detail[4]], -1, (0, 255, 0), 2, cv2.LINE_AA)
        
            

capture = cv2.VideoCapture(0)

while capture.isOpened():
    success, frame = capture.read()
    if(success):
        getContours(frame, (40, 140), 40000)
        cv2.imshow("Frame", frame)
    if(cv2.waitKey(40) == 27):
        break
    
capture.release()
cv2.destroyAllWindows()
    